In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, VarianceThreshold, SelectFromModel
from sklearn.feature_selection import mutual_info_regression, f_regression
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,DotProduct, ConstantKernel,WhiteKernel,Matern,RationalQuadratic

# Preprocessing
## 1. Read data from csv file
Use pandas to read csv file. Then discard unnecessary columns (id column). Check the correctness of data reading in the end of the cell.

In [2]:
x_train = pd.read_csv("task1/X_train.csv")
y_train = pd.read_csv("task1/Y_train.csv")
x_test = pd.read_csv("task1/X_test.csv")
# remove the id column of x_train and x_test
x_train = x_train.iloc[:, 1:]
x_test = x_test.iloc[:, 1:]
# remove the id column of y_train
# can also use drop() function
y_train = y_train.iloc[:,1:]

# check whether read data correctly
# print(x_train.shape)
x_train.head(7)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
0,10.891876,832442.812375,20585.544083,1028.369495,1.163780e+06,9.199135,597900.477629,NaN,1.144294e+06,785176.201298,...,1.024198e+06,-855.549602,12176.073427,10.647729,10.916371,1220.065443,8.566724,1.036263e+06,85338.558539,103088.664210
1,11.512994,832442.898114,NaN,1012.624877,1.028911e+06,10.906408,597900.458612,8127.016078,1.099166e+06,785176.258299,...,1.086806e+06,-787.397942,10493.095660,10.586492,9.463962,917.094909,10.231822,1.007163e+06,95695.020645,105161.109422
2,11.052185,832442.896307,20585.512844,1003.953827,9.231756e+05,9.212979,597900.426764,10738.092422,1.027863e+06,785176.223468,...,1.018533e+06,-906.997242,10959.516944,10.769287,10.342160,637.027802,10.705461,1.019955e+06,80253.299882,104177.051666
3,11.642076,NaN,NaN,1004.672084,9.459461e+05,9.553420,597900.450367,13524.096973,1.168144e+06,785176.254867,...,1.047017e+06,-1011.742516,16845.309819,10.483830,10.594941,1114.069590,10.321063,1.085442e+06,NaN,102746.516920
4,10.407121,832442.831424,20585.557007,NaN,9.957182e+05,8.419164,597900.423639,12894.065081,1.063199e+06,785176.190880,...,1.031009e+06,-1025.223865,18348.460040,NaN,NaN,1230.088215,10.250096,1.024812e+06,101815.745499,105163.749149
5,9.144461,832442.882921,20585.548624,1042.982264,1.079217e+06,10.290930,597900.434742,9730.044411,9.758330e+05,785176.248738,...,1.020202e+06,-790.713766,12838.791399,10.561760,10.386269,1230.071653,11.034434,NaN,90435.964659,109082.145240
6,9.895803,832442.816841,20585.544187,1001.483820,9.935226e+05,10.276043,597900.436187,12080.006126,9.537706e+05,785176.270535,...,1.038143e+06,-927.126335,14383.974419,10.427384,10.442063,1076.013807,10.228285,1.054731e+06,100889.137964,106036.091106


## 2. Filling the missing values
After reading data, we could see there are many missing values('NAN') in the dataset. So before we do further processing, we should choose proper methods to fill the missing values.    

Mean value of each column or Median value of each column can be used to do the fulfillment. But here we will take KNN methods to get more accurate outcome.  
  
  Use *KNNImputer* from *sklearn.impute* to fill the missing values('NAN').

In [3]:
# imputer = KNNImputer( n_neighbors=10, weights='uniform', metric='nan_euclidean')

# x_train = imputer.fit_transform(x_train)
# x_train = pd.DataFrame(x_train)
# x_test = imputer.fit_transform(x_test)
# x_test = pd.DataFrame(x_test)

# x_train = x_train.fillna(x_train.mean())
# x_test = x_test.fillna(x_train.mean())
x_train = x_train.fillna(x_train.median())
x_test = x_test.fillna(x_train.median())

## 3. Scaling the data
Each column's data varies a lot. To take all data as same weight(since we do not know which columns are useful yet), we should scale the data: mapping each column value to a small range.  
  
  Use MinMaxScaler and StandardScaler from sklearn.preprocessing to do this. We choose to use StandardScaler in our code by trying both.

In [5]:
# select scaling method
scaler = preprocessing.StandardScaler()
# scaler = preprocessing.MinMaxScaler(feature_range = (0,1))
x_train_scaled = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train_scaled, columns = x_train.columns)
x_test_scaled = scaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test_scaled, columns = x_test.columns)

# check the data's correctness
x_train.head(7)

,x7,x10,x12,x22,x30,x35,x52,x59,x62,x66,...,x780,x783,x785,x786,x789,x792,x800,x803,x819,x827
0,-0.055135,-1.277041,-1.196397,0.262751,-0.754146,0.778379,-0.982276,1.097465,1.577344,-1.092229,...,-0.791156,-1.536247,1.433454,0.580604,-0.697564,0.196070,-0.053570,-1.808424,-0.407864,0.716959
1,-1.413360,1.078990,1.437455,-0.839554,-2.357243,-0.219447,0.055632,-0.082797,-0.126604,-0.595918,...,0.200293,1.441818,1.195678,-2.790419,-1.396350,-4.312009,-2.749840,1.922087,-2.693170,-0.681028
2,0.222449,0.405424,-1.001573,-0.610272,1.663599,-0.822858,0.970240,0.506651,-0.793479,-0.069854,...,0.354999,-1.120218,-0.015314,-0.523713,1.217296,2.167882,0.219672,-0.499415,1.230026,-1.973333
3,1.967849,1.545843,0.696952,0.437450,2.206252,1.001727,1.983694,2.761847,0.439390,2.215155,...,-0.045700,1.469567,-0.879843,2.637560,0.693905,2.308818,1.584764,1.014288,2.379181,0.227866
4,1.573141,1.445035,1.134607,0.096823,0.075987,0.752605,0.756024,-0.271707,1.815611,0.376753,...,1.934263,0.521540,0.514607,-0.134834,-0.026744,-0.032662,0.208839,0.418259,0.251549,0.763207
5,-0.409081,0.926110,0.514788,1.714245,-0.594199,0.086438,0.150011,-1.298342,1.500214,0.293366,...,-0.068983,-0.664400,1.081272,-0.137400,-0.046180,-0.357644,0.096074,-0.163661,0.044461,0.763131
6,1.063142,1.590888,1.032933,2.327442,0.489721,0.837099,0.316016,-0.845485,-0.513674,1.237485,...,0.847885,1.607146,1.429756,1.015503,0.706206,0.631430,0.706767,1.038476,0.303295,0.052266


## 4. Feature Selection using Lasso
  
  There are 832 features(columns) of each data. Among them exists useless or redundant features.Too many features may cause problems such as low-effiency or overfit. It's necessary to reduce features' number.   
  Here are several methods to select features:
  - Filter
  - Wrapper
  - Embedded 
  
<br>We first tried SelectKBest method from sklearn but it doesn't work (cannot distinguish which are useful features because it may choose relevant features and cause overfit). So in our code we finally choose Lasso to pick up features.
<br><br>Lasso methods fit such problems: small sample but relatively many features. L1 regularization adds the L1 norm of the coefficient W to the loss function as a penalty term. Since the regular term is non-zero, this forces the coefficients corresponding to weak features to be zero. Thus L1 regularization tends to make the learned model sparse (the coefficient W is often 0). That will meet our requirements. 

In [ ]:
# alpha: the weight of L1 penalty term
coefficients_train = Lasso(alpha = 0.42)
coefficients_train.fit(x_train, y_train)

choose_features = (coefficients_train.coef_ != 0)
print(coefficients_train.coef_)
print(choose_features)
x_train = x_train.loc[:, choose_features]
x_test = x_test.loc[:, choose_features]

#check choose how many features
x_train.head()

## 4.Feature Selection using Random Forest

A random forest is a machine learning technique that’s used to solve regression and classification problems. It utilizes ensemble learning, which is a technique that combines many classifiers to provide solutions to complex problems.

<br>A random forest algorithm consists of many decision trees. The ‘forest’ generated by the random forest algorithm is trained through bagging or bootstrap aggregating. Bagging is an ensemble meta-algorithm that improves the accuracy of machine learning algorithms.

In [4]:
sel = RandomForestRegressor()
sel.fit(x_train, y_train)
np.sort(sel.feature_importances_)[630:]
# sfm = SelectFromModel(sel, threshold = 0.00135993)
sfm = SelectFromModel(sel, threshold = 0.00095)
sfm.fit(x_train, y_train)
coefficients_train = sfm.get_support()
choose_features = (coefficients_train == True)
print(choose_features)
x_train = x_train.loc[:, choose_features]
x_test = x_test.loc[:, choose_features]

#check choose how many features
x_train.head()

<ipython-input-4-7ca8ed68f2a5>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  sel.fit(x_train, y_train)
/usr/local/anaconda3/lib/python3.8/site-packages/sklearn/feature_selection/_from_model.py:234: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


[False False False False False False False  True False False  True False
  True False False False False False False False False False  True False
 False False False False False False  True False False False False  True
 False False False False False False False False False False False False
 False False False False  True False False False False False False  True
 False False  True False False False  True  True  True False False False
 False False False False False False False False False  True False False
 False  True False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False  True False
 False  True False False False False False  True False  True False False
 False False False False False False False False False False  True False
 False False  True False False False False False False False False  True
 False False False False False False False False Fa

,x7,x10,x12,x22,x30,x35,x52,x59,x62,x66,...,x780,x783,x785,x786,x789,x792,x800,x803,x819,x827
0,10295.013382,38042.008654,67475.900839,2538.045240,1.965179,5555.414472,2.075892,2.876462,2888.814498,5421.067346,...,5008.077457,16053.672538,2457.075229,2.328664,1.836302,2.586985,2.358928,309702.424398,2.139276,1220.065443
1,8127.016078,50949.420639,93179.779900,2033.004526,1.716548,4784.223551,2.244688,2.576871,1793.052899,5820.082835,...,6052.094261,23841.492645,2295.387016,1.847425,1.737345,1.796449,1.886463,557053.464687,1.746126,917.094909
2,10738.092422,47259.321206,69377.195188,2138.054129,2.340156,4317.864410,2.393432,2.726493,1364.203740,6243.017475,...,6215.003373,17141.615450,1471.906345,2.171014,2.107470,2.932762,2.406808,396496.091177,2.421048,637.027802
3,13524.096973,53507.052907,85953.175663,2618.086812,2.424319,5728.033993,2.558251,3.298939,2157.028191,8080.077740,...,5793.057983,23914.059050,884.023158,2.622310,2.033351,2.957477,2.646011,496861.936593,2.618742,1114.069590
4,12894.065081,52954.782341,90224.274392,2462.022541,2.093927,5535.494157,2.358593,2.528920,3042.037387,6602.072854,...,7878.002747,21434.911682,1832.255623,2.226530,1.931298,2.546875,2.404909,457342.316287,2.252717,1230.088215


## 5.Outlier detection
<br>Use IsolationForest from sklearn to do isolation forest methods. Detect and remove  outliers.

In [6]:
# create IsolationForest class
isofore = IsolationForest(n_estimators = 600, max_features = 29, contamination = 0.015)

"""
IsolationForest(*, n_estimators=100, max_samples='auto', contamination='auto', max_features=1.0, bootstrap=False, n_jobs=None, random_state=None, verbose=0, warm_start=False)
"""
x_train_isoforest = isofore.fit_predict(x_train)

# Remove outliers
x_train = x_train[(x_train_isoforest != -1)]
y_train = y_train[(x_train_isoforest != -1)]

# print(x_train)

# Regression using XGBoost
<br> Use processed data to train and predict the 'X_test.csv'.

In [ ]:
# Step 5 - Model Fit: XGBoost

regression = xgb.XGBRegressor(objective="reg:squarederror", random_state=48)

# Use 5-fold Cross Validation to See the performance and tune the parameter

cv_means = []
cv_stds = []
for i in np.arange(10):
    scores = cross_val_score(estimator = regression,
                                 X = x_train,
                                 y = y_train,
                                 scoring = 'r2',
                                 cv = KFold(n_splits = 5, shuffle = True))
    cv_means.append(np.mean(scores))
    cv_stds.append(np.std(scores))

print("Average of R2 scores:", np.mean(cv_means))
print("Standard deviation of R2 scores:", np.mean(cv_stds))

# Regression using Gaussian

In [7]:
# Step 5 Candidate - Model Fit: Gaussian Regression
# kl = 1 * RationalQuadratic() + WhiteKernel(noise_level_bounds=(1e-20, 1e1))
kl = 1 * RationalQuadratic()
# kl= 1.0 * RBF(length_scale=100.0, length_scale_bounds=(1e-2, 1e3)) + WhiteKernel(
#     noise_level=1, noise_level_bounds=(1e-10, 1e1)
# )
regression_gaussian = GaussianProcessRegressor(kernel=kl, n_restarts_optimizer=3, normalize_y=True)
# regression_gaussian.fit()
cv_means = []
cv_stds = []
# changed to 10-fold
# changed to 5-fold
for i in np.arange(10):
    scores = cross_val_score(estimator = regression_gaussian,
                                 X = x_train,
                                 y = y_train,
                                 scoring = 'r2',
                                 cv = KFold(n_splits = 5, shuffle = True))
    cv_means.append(np.mean(scores))
    cv_stds.append(np.std(scores))

print("Average of R2 scores:", np.mean(cv_means))
print("Standard deviation of R2 scores:", np.mean(cv_stds))

Average of R2 scores: 0.6707510394109997
Standard deviation of R2 scores: 0.028824075643680745


# Prediction and Write into csv file

In [8]:
# regression = xgb.XGBRegressor(objective = "reg:squarederror", random_state = 48)
# regression.fit(x_train, y_train)
# y_pred = regression.predict(x_test)

regression_gaussian.fit(x_train, y_train)
y_pred = regression_gaussian.predict(x_test)

print(y_pred)
prediction_results = pd.DataFrame(data = y_pred,columns = ['y'])
# Using DataFrame.insert() to add a column
index = [i for i in range(len(prediction_results))]
prediction_results.insert(0,"id",index)
  
# Observe the result
prediction_results
prediction_results.to_csv('task1/result_forest_5.csv',index = False)

[[70.23807521]
 [72.80924118]
 [70.53185577]
 [67.28527531]
 [67.59217519]
 [80.5236718 ]
 [61.82554197]
 [62.53339349]
 [80.00086314]
 [77.226565  ]
 [54.10925582]
 [88.23042863]
 [69.06055672]
 [81.33211377]
 [55.58738031]
 [90.31406167]
 [73.57357423]
 [75.84052981]
 [71.59255338]
 [72.82161562]
 [69.97773321]
 [63.98872163]
 [79.47612833]
 [78.00247325]
 [59.0049291 ]
 [72.34280527]
 [58.51905438]
 [71.51947643]
 [59.40017238]
 [74.52459488]
 [67.27320244]
 [71.29049092]
 [60.35343864]
 [72.45592124]
 [69.08578533]
 [61.9074445 ]
 [73.89150474]
 [51.44192609]
 [75.17312364]
 [69.21099521]
 [65.37089671]
 [75.83460041]
 [72.34306894]
 [80.35538042]
 [74.15873946]
 [66.44882652]
 [69.56098473]
 [72.33857818]
 [71.92746321]
 [67.4004561 ]
 [76.81071437]
 [63.47141918]
 [63.55296903]
 [82.6167046 ]
 [82.35408977]
 [73.27119672]
 [60.23156311]
 [70.50946472]
 [71.98381888]
 [59.13254253]
 [59.59234627]
 [73.54357614]
 [62.20169476]
 [73.22907874]
 [70.93170718]
 [57.28856663]
 [64.04704